In [1]:
# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine

# Database connection function
def connect_to_sakila_db(user, password, host, port, db_name):
    try:
        engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_name}')
        connection = engine.connect()
        print("Connection to the Sakila database was successful.")
        return engine
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

# Function to retrieve rental data for a specific month and year
def rentals_month(engine, month, year):
    query = f'''
    SELECT customer_id, COUNT(rental_id) as rentals
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id
    '''
    try:
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

# Function to get rental count per customer for a specific month and year
def rental_count_month(df, month, year):
    column_name = f"rentals_{str(month).zfill(2)}_{year}"
    df[column_name] = df['rentals']
    df = df.drop('rentals', axis=1)
    return df

# Function to compare rentals across two months and calculate the difference
def compare_rentals(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    rental_cols = [col for col in merged_df.columns if col.startswith('rentals_')]
    if len(rental_cols) == 2:
        merged_df['difference'] = merged_df[rental_cols[1]] - merged_df[rental_cols[0]]
    return merged_df

In [2]:
engine = connect_to_sakila_db('user', 'password', 'localhost', 3306, 'sakila')
print(engine)


Error connecting to database: (pymysql.err.OperationalError) (1045, "Access denied for user 'user'@'localhost' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)
None


In [4]:
engine = connect_to_sakila_db('root', 'Ss871921%', 'localhost', 3306, 'sakila')


Connection to the Sakila database was successful.


In [6]:
may_rentals = rentals_month(engine, 5, 2005)
print(may_rentals.head())
(may_rentals.info())  #  data types and structure


   customer_id  rentals
0            1        2
1            2        1
2            3        2
3            5        3
4            6        3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   customer_id  520 non-null    int64
 1   rentals      520 non-null    int64
dtypes: int64(2)
memory usage: 8.3 KB


In [7]:
may_count = rental_count_month(may_rentals, 5, 2005)
print(may_count.head())


   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [8]:
june_rentals = rentals_month(engine, 6, 2005)
june_count = rental_count_month(june_rentals, 6, 2005)
print(june_count.head())
print(june_count.info())  #  data types and structure

   customer_id  rentals_06_2005
0            1                7
1            2                1
2            3                4
3            4                6
4            5                5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   customer_id      590 non-null    int64
 1   rentals_06_2005  590 non-null    int64
dtypes: int64(2)
memory usage: 9.3 KB
None


In [9]:
comparison = compare_rentals(may_count, june_count)
print(comparison.head())


   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1              2.0              7.0         5.0
1            2              1.0              1.0         0.0
2            3              2.0              4.0         2.0
3            4              0.0              6.0         6.0
4            5              3.0              5.0         2.0
